## Imports

In [1]:
import sqlite3
import pandas as pd
from sqlite3 import Error

## Functions

In [2]:

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print('Version: ', sqlite3.version)
        print('---------------\n')
    except Error as e:
        print(e)
 
    return conn


def fetch_all_tables(conn):      
    '''
    prints out all of the columns of every table in databse
    cur : cursor object
    conn : database connection object
    '''
    cur = conn.cursor()

    query = """
        SELECT name
        FROM sqlite_master
        WHERE type='table';
        """

    cur.execute(query)
    tables = cur.fetchall()
    for table_name in tables:
        table_name = table_name[0] # tables is a list of single item tuples
        table = pd.read_sql_query("SELECT * from {} LIMIT 0".format(table_name), conn)
        print(table_name)
        for col in table.columns:
            print('\t' + col)
        print()

         
def sqlite_query(query, conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    table = pd.read_sql_query(query, conn)
    display(table)

## Main

In [3]:
database = "sqlite_db_pythonsqlite.db"

# create a database connection
conn = create_connection(database)

#Display tables and schemas
with conn: 
    print("=== TABLES ===")
    fetch_all_tables(conn)

Version:  2.6.0
---------------

=== TABLES ===
Bookings
	bookid
	facid
	memid
	starttime
	slots

Facilities
	facid
	name
	membercost
	guestcost
	initialoutlay
	monthlymaintenance

Members
	memid
	surname
	firstname
	address
	zipcode
	telephone
	recommendedby
	joindate



In [4]:
with conn: 
    query = """
    SELECT DATE(MIN(starttime)) AS start_date, DATE(MAX(starttime)) AS end_date
    FROM Bookings
    """
    sqlite_query(query, conn)

,start_date,end_date
0,2012-07-03,2012-09-30


In [5]:
with conn:
    description = 'A list of facilities with a total revenue less than 1000.\n' +\
    'The output of facility name and total revenue, sorted by revenue:'
    print(description)
    
    query = """
    SELECT name, SUM(sales) AS total_revenue FROM
        (SELECT name,
        CASE
            WHEN `memid` = 0 THEN slots*guestcost
            ELSE slots*membercost
            END AS sales
        FROM `Bookings` AS b
        JOIN 'Facilities' AS f
        ON b.facid = f.facid) AS sales_data
    GROUP BY name
    HAVING total_revenue < 1000
    ORDER BY total_revenue
    ;
    """
    sqlite_query(query, conn)

A list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue:


,name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


In [6]:
with conn:
    description = 'Produce a report of members and who recommended them in alphabetic surname,firstname order:'
    print(description)
    
    query = """
    SELECT a.surname, a.firstname, (b.firstname|| ' ' ||b.surname) AS recommender
    FROM `Members` AS a
    INNER JOIN `Members` AS b
    ON a.recommendedby = b.memid
    
    UNION
    
    SELECT c.surname, c.firstname, Null
    FROM `Members` AS c
    WHERE c.recommendedby = ""
    ORDER BY a.surname, a.firstname
    
    """
    sqlite_query(query, conn)

Produce a report of members and who recommended them in alphabetic surname,firstname order:


,surname,firstname,recommender
0,Bader,Florence,Ponder Stibbons
1,Baker,Anne,Ponder Stibbons
2,Baker,Timothy,Jemima Farrell
3,Boothe,Tim,Tim Rownam
4,Butters,Gerald,Darren Smith
5,Coplin,Joan,Timothy Baker
6,Crumpet,Erica,Tracy Smith
7,Dare,Nancy,Janice Joplette
8,Farrell,David,None
9,Farrell,Jemima,None


In [7]:
with conn:
    description = 'Find the facilities with their usage by member, but not guests:'
    print(description)
    
    query = """
    SELECT Facilities.name as facility, grouped_facility.total_reservation_slots, grouped_facility.total_usage_hours
    FROM    
        (SELECT facid, SUM(sum_slots) AS total_reservation_slots, SUM(sum_slots) *0.5 AS total_usage_hours
        FROM
            (SELECT facid, memid, SUM(slots) AS sum_slots
            FROM Bookings
            GROUP BY facid, memid
            HAVING memid != 0) AS grouped_user
        GROUP BY facid) AS grouped_facility
    JOIN Facilities
    ON Facilities.facid = grouped_facility.facid
    ORDER BY total_usage_hours DESC
    """
    sqlite_query(query, conn)

Find the facilities with their usage by member, but not guests:


,facility,total_reservation_slots,total_usage_hours
0,Badminton Court,1086,543.0
1,Tennis Court 1,957,478.5
2,Massage Room 1,884,442.0
3,Tennis Court 2,882,441.0
4,Snooker Table,860,430.0
5,Pool Table,856,428.0
6,Table Tennis,794,397.0
7,Squash Court,418,209.0
8,Massage Room 2,54,27.0


In [8]:
with conn:
    description = 'Find the facilities with their usage by member, but not guests:'
    print(description)
    
    query = """
    SELECT Facilities.name AS facility, reservation_month, reservation_slots, reservation_slots * 0.5 as usage_hours
    FROM
    (SELECT facid,
        strftime("%m-%Y", starttime) as reservation_month,
        SUM(slots) AS reservation_slots
    FROM Bookings
    WHERE memid != 0
        GROUP BY facid, strftime("%m-%Y", starttime)) as group_by_month
    JOIN Facilities
    ON Facilities.facid = group_by_month.facid
    """
    sqlite_query(query, conn)

Find the facilities with their usage by member, but not guests:


,facility,reservation_month,reservation_slots,usage_hours
0,Tennis Court 1,07-2012,201,100.5
1,Tennis Court 1,08-2012,339,169.5
2,Tennis Court 1,09-2012,417,208.5
3,Tennis Court 2,07-2012,123,61.5
4,Tennis Court 2,08-2012,345,172.5
5,Tennis Court 2,09-2012,414,207.0
6,Badminton Court,07-2012,165,82.5
7,Badminton Court,08-2012,414,207.0
8,Badminton Court,09-2012,507,253.5
9,Table Tennis,07-2012,98,49.0
